In [10]:
import pandas as pd
import plotly.io as pio
import plotly.express as px

# Benutzerdefiniertes Template definieren
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            y=0.91,  # Titel weiter nach oben
            x=0.05,  # Links ausgerichtet
            xanchor="left",  # Linksbündig
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

### load the data summed up

In [11]:
years = [str(year) for year in range(2013, 2025)]

df = pd.read_excel("data.xlsx", sheet_name=years)

data_list = []

for year, df in df.items():
    # Füge das Jahr als neue Spalte hinzu
    df['Jahr'] = year
    
    # Füge den DataFrame der Liste hinzu
    data_list.append(df)

# Alle DataFrames in der Liste zu einem einzigen DataFrame zusammenführen
final_df = pd.concat(data_list, ignore_index=True)



### clean the data

In [12]:
df = final_df[final_df.columns[~final_df.columns.str.contains(r'\*')]] # drop * in the data
df.columns = df.columns.str.replace('*', '', regex=False)# just to be sure
df = df.rename(columns={'Master Getränketechnologi':'Master Getränketechnologie'})# fix a typo
clean_columns = {}
for col in df.columns:
    clean_name = col.strip()  # get rid of other human typos
    if clean_name in clean_columns:
        clean_columns[clean_name].append(col)
    else:
        clean_columns[clean_name] = [col]

# smack together the similar col names 
merged_df = df[['Jahr']].copy()
for clean_name, original_cols in clean_columns.items():
    if len(original_cols) > 1:
        merged_df[clean_name] = df[original_cols].sum(axis=1)
    else:
        merged_df[clean_name] = df[original_cols[0]]
df = merged_df.fillna(value=0) # use if needed for plotting

### transform

In [13]:
df = df[df['Variable'].isin(['Zufriedenheit'])]
df = df[df['Variable'] == 'Zufriedenheit'].drop(columns='Variable')
df = df[df["Category"] != "Gesamt (%)"]
df = df.groupby(['Jahr', 'Category']).sum().reset_index()


In [14]:
anzahl_row = df[df['Category'] == 'Anzahl']
for year in df['Jahr'].unique():
    gesamtzahlen = anzahl_row[anzahl_row['Jahr'] == year].drop(columns=['Jahr', 'Category']).values.flatten()

    # Sicherstellen, dass alle relevanten Spalten als numerische Werte vorliegen
    for col in df.columns[2:]:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Umwandlung in numerisch, fehlerhafte Werte werden zu NaN

    # Gehe alle Zeilen durch, die nicht 'Anzahl' sind
    for idx, row in df[df['Jahr'] == year].iterrows():
        if row['Category'] != 'Anzahl':
            # Extrahiere die Prozentsätze der aktuellen Zeile
            relative_prozent = row.drop(['Jahr', 'Category']).values
            # Berechne die absoluten Werte
            absolute_werte = (relative_prozent / 100) * gesamtzahlen
            # Überschreibe die Prozentsätze mit den berechneten absoluten Werten
            df.loc[idx, df.columns[2:]] = absolute_werte
            df.round(0)
df = df[df['Category'] != 'Anzahl']
df['sum']= df.sum(axis=1, numeric_only=True).round(0)
df = df.get(key=['Jahr', 'Category', 'sum'])

In [16]:
category_order = [
    'gar nicht zufrieden (%)', 
    'weniger zufrieden (%)', 
    'mittelmäßig zufrieden (%)', 
    'eher zufrieden (%)', 
    'sehr zufrieden (%)'
]
df['Category'] = pd.Categorical(df['Category'], categories=category_order, ordered=True)

In [23]:
df

,Jahr,Category,sum
2,2013,gar nicht zufrieden (%),2.0
5,2013,weniger zufrieden (%),85.0
3,2013,mittelmäßig zufrieden (%),67.0
1,2013,eher zufrieden (%),393.0
4,2013,sehr zufrieden (%),22.0
8,2014,gar nicht zufrieden (%),4.0
11,2014,weniger zufrieden (%),115.0
9,2014,mittelmäßig zufrieden (%),46.0
7,2014,eher zufrieden (%),357.0
10,2014,sehr zufrieden (%),21.0


In [18]:
category_order = [
    'gar nicht zufrieden (%)', 
    'weniger zufrieden (%)', 
    'mittelmäßig zufrieden (%)', 
    'eher zufrieden (%)', 
    'sehr zufrieden (%)'
]

# Stelle sicher, dass 'Category' eine kategorische Variable mit der richtigen Reihenfolge ist
df['Category'] = pd.Categorical(df['Category'], categories=category_order, ordered=True)

# Sortiere den DataFrame nach Jahr und Category
df = df.sort_values(by=['Jahr', 'Category'])

fig = px.bar(df, x="Jahr", 
             y="sum", 
             color="Category", 
             title="Zufriedenheit mit dem Studium im Fachbereich", 
             color_discrete_sequence=px.colors.qualitative.Safe, 
             template="infoviz", 
             labels={
                 'Jahr': 'Jahr', 
                 'sum': 'Beteiligung an der Befragung',
                 'Category': 'Kategorien'
             })
fig.show()